In [1]:
from models.resnet.resnet_se_34v2 import ResNetSE34V2
from torchaudio.transforms import MelSpectrogram
from torchaudio.functional import amplitude_to_DB
import torch
import torchaudio
import librosa
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import yaml
import random

In [2]:
DATASET_PATH = "IEMOCAP_full_release"
PATH = os.path.join(os.getcwd(), DATASET_PATH)

In [3]:
data = []
spk_M = 0
spk_F = 0

for i, subdir in enumerate(os.listdir(PATH)):


    if (subdir[0] != '.'):
        #for j in random.sample(range(0, 20), 2):
        #for i, file in enumerate(os.listdir(os.path.join(PATH, subdir)+'/sentences/wav')):
        file = os.listdir(os.path.join(PATH, subdir)+'/sentences/wav')
        file_M = [path for path in file if path[5] == 'M']
        file_F = [path for path in file if path[5] == 'F']

        #Choose 2 random file in a directory 
        for j in random.sample(range(0, 10), 2):
            
      
            file2 = os.listdir(os.path.join(PATH, subdir)+'/sentences/wav/'+file_M[j]) 

            file2_M = [path for path in file2 if path[-8] == 'M']
            file2_F = [path for path in file2 if path[-8] == 'F']

            for k in random.sample(range(0, 10), 5):


                sample_M = os.path.join(PATH, subdir)+'/sentences/wav/'+file_M[j] +'/'+ file2_M[k]
                data.append([sample_M, "Spk_"+str(2*int(subdir[-1])-1)])
                sample_F = os.path.join(PATH, subdir)+'/sentences/wav/'+file_M[j] +'/'+ file2_F[k]
                data.append([sample_F, "Spk_"+str(2*int(subdir[-1]))])


            file2 = os.listdir(os.path.join(PATH, subdir)+'/sentences/wav/'+file_F[j])

            file2_M = [path for path in file2 if path[-8] == 'M']
            file2_F = [path for path in file2 if path[-8] == 'F']

            for k in random.sample(range(0, 10), 5):


                sample_M = os.path.join(PATH, subdir)+'/sentences/wav/'+file_F[j] +'/'+ file2_M[k]
                data.append([sample_M, "Spk_"+str(2*int(subdir[-1])-1)])
                sample_F = os.path.join(PATH, subdir)+'/sentences/wav/'+file_F[j] +'/'+ file2_F[k]
                data.append([sample_F, "Spk_"+str(2*int(subdir[-1]))])

In [4]:
print(len(data))

200


In [5]:
num_spk = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 
           7:0, 8:0, 9:0}

for i in data : 
    num_spk[int(i[1][-1])] += 1

print(num_spk)

{0: 20, 1: 20, 2: 20, 3: 20, 4: 20, 5: 20, 6: 20, 7: 20, 8: 20, 9: 20}


In [6]:
def load_audio(file):
    EPS = 1e-8
    s, _ = librosa.load(file, sr=16000)
    amax = np.max(np.abs(s))
    factor = 1.0 / (amax + EPS)
    s = s * factor
    return s

In [7]:
with open('./models/resnet/config.yaml') as f:
    config = yaml.safe_load(f)

sd = torch.load('./models/weigths/resnetse34_epoch92_eer0.00931.pth')
model = ResNetSE34V2(nOut=256, n_mels= 80)
model.load_state_dict(sd)
model.eval()
torch.set_grad_enabled(False)

transform = MelSpectrogram(
    sample_rate= 16000,
    n_fft= 512,
    win_length= 400,
    hop_length= 160,
    window_fn=torch.hamming_window,
    n_mels= 80,
    f_min=20,
    f_max=7600,
    norm='slaney')


def embed_inference(audio_path, transform= transform, model = model):
    s = load_audio(audio_path)
    #print(type(s))
    x = torch.from_numpy(s[None, :])
    x = transform(x)
    x = amplitude_to_DB(
        x, multiplier=10, amin= 1e-5, db_multiplier=0, top_db=75)

    feature = model(x[:, None, :, :])
    feature = torch.nn.functional.normalize(feature)
    return(feature,s)

Embedding size is 256, encoder SAP.


# PROCESS THE TEST

In [8]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/.test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


In [9]:
from speechbrain.pretrained import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")


In [10]:
from modelscope.models import Model
from modelscope.pipelines import pipeline

model = Model.from_pretrained('damo/speech_campplus_sv_en_voxceleb_16k', device='cpu')


2023-10-11 19:26:49,417 - modelscope - INFO - PyTorch version 2.1.0 Found.
2023-10-11 19:26:49,418 - modelscope - INFO - Loading ast index from /Users/hugo/.cache/modelscope/ast_indexer
2023-10-11 19:26:49,472 - modelscope - INFO - Loading done! Current index file version is 1.9.2, with md5 1a341cbc642fe5a28627294590b041fb and a total number of 941 components indexed


MODEL CONFIG :  modelscope/1.9.2; python/3.8.18; session_id/61c54b3edbd54e49a8d63cd67ff19afc; platform/macOS-13.3-x86_64-i386-64bit; processor/i386; env/custom; user/unknown; invoked_by/from_pretrained


2023-10-11 19:26:52,337 - modelscope - INFO - Model revision not specified, use revision: v1.0.2
2023-10-11 19:26:53,344 - modelscope - INFO - initialize model from /Users/hugo/.cache/modelscope/hub/damo/speech_campplus_sv_en_voxceleb_16k


In [11]:
sim = torch.nn.CosineSimilarity(dim=1, eps=1e-8)

In [12]:
print(data)

[['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_M027.wav', 'Spk_7'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_F013.wav', 'Spk_8'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_M026.wav', 'Spk_7'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_F017.wav', 'Spk_8'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_impro02_M024.wav', 'Spk_7'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04M_impro02/Ses04M_im

In [14]:

compar_tab = pd.DataFrame(columns = ['spk_1', 'spk_2', 'duration_1', 'duration_2', 'score_classic', 'score_CAM++', 'TDN_score','path_1', 'path_2'])
already_done = []
with tqdm(total=len(data)*len(data)) as pbar:
    for sample_1 in data :
        spk_1 = sample_1[1]
        ref_embed, _ = embed_inference(sample_1[0])

        audio_1, _ = torchaudio.load(sample_1[0])
        for sample_2 in data :

            if {sample_1[0],sample_2[0]} not in already_done :
                #print('test')
                spk_2 = sample_2[1]

                
                embed, _ = embed_inference(sample_2[0])
                audio_2, _ = torchaudio.load(sample_2[0])
                loss_val = sim(ref_embed, embed)

                #print(audio_1.shape, audio_1)
                embeddings = model(audio_1)
                embeddings2 = model(audio_2)
                cam_score_model = sim(embeddings, embeddings2)
                



                TDN_score = verification.verify_files(sample_1[0], sample_2[0]) # Different Speakers
                compar_tab.loc[len(compar_tab)] = [spk_1, spk_2, len(audio_1)/16000, len(audio_2)/16000, loss_val.item(),cam_score_model, TDN_score, sample_1[0], sample_2[0]]

                already_done.append({sample_1[0],sample_2[0]})
                #print(label, val)
            pbar.update(1)
       

           
compar_tab.to_csv('compar_tab_2.csv', index=False)

100%|██████████| 40000/40000 [9:16:46<00:00,  1.20it/s]   


In [15]:
import numpy as np
from sklearn.metrics import roc_curve, auc

def calculate_eer_and_fpr_fnr(scores, labels):
    fpr, tpr, thresholds = roc_curve(labels, scores)
    fnr = 1 - tpr

    eer_threshold = thresholds[np.nanargmin(np.abs(fnr - fpr))]
    eer = fnr[np.nanargmin(np.abs(fnr - fpr))]

    fpp = np.sum((scores >= eer_threshold) & (labels == 0))
    fnp = np.sum((scores < eer_threshold) & (labels == 1))
    total_negatives = np.sum(labels == 0)
    total_positives = np.sum(labels == 1)

    print(labels)
    print(scores)
    return eer, fpp, fnp, total_positives, total_negatives

# Replace 'verification_scores' and 'verification_labels' with your actual data


# print(f"Equal Error Rate (EER): {eer * 100:.2f}%")
# print(f"False Positive Predictions (FPP): {fpp}")
# print(f"False Negative Predictions (FNP): {fnp}")
# print(f"Total Positives: {total_positives}")
# print(f"Total Negatives: {total_negatives}")


******************************************************************
********************** CHOSEN TRESHOLD :  0.2  **********************


IndexError: list index out of range

In [37]:
text = 'tensor([1.0000])'

In [16]:
# How to change a str written like {'score':1.0 , 'text':'hello'} to a dict
def str_to_dict_cam(input : str):
    input = input.replace(' ', '')
    input = input.replace('tensor', '')
    input = input.replace('(', '')
    input = input.replace(')', '')
    input = input.replace("[", '')
    input = input.replace("]", '')


    return input

In [17]:
print(str_to_dict_cam(text))

NameError: name 'text' is not defined

In [18]:
def str_to_dict_TDN(input : str):
    input = input.replace('tensor', '')
    input = input.replace(' ', '')
    input = input.replace('{', '')
    input = input.replace('}', '')
    input = input.replace("'", '')
    input = input.replace("(", '')
    input = input.replace(")", '')
    input = input.replace('[', '')
    input = input.replace(']', '')

    output = input.split(',')

    return output

In [26]:
df = pd.read_csv('compar_tab_2.csv')

false_positive_classic = 0
false_positive_CAM = 0
false_negative_classic = 0
false_negative_CAM = 0
false_positive_TDN = 0
false_negative_TDN = 0
positive = 0
negative = 0
treshold = 0.3

for treshold in [0.1,0.15,0.2,0.25,0.3,0.4,0.5,0.6,0.7] :
    print('********************** CHOSEN TRESHOLD : ', treshold, ' **********************')
    for row, line in df.iterrows():
        score_classic = line['score_classic']
    
        CAM = str_to_dict_cam(line['score_CAM++'])
        score_CAM = float(CAM)
        
        TDN = str_to_dict_TDN(line['TDN_score'])
        score_TDN = float(TDN[0])

        if line['spk_1'] == line['spk_2'] :
            positive += 1
            if score_classic < treshold :
                false_positive_classic += 1
            if score_CAM < treshold :
                false_positive_CAM += 1
            if score_TDN < treshold :
                false_positive_TDN += 1
        else :
            negative += 1
            if score_classic > treshold :
                false_negative_classic += 1
            if score_CAM > treshold :
                false_negative_CAM += 1
            if score_TDN > treshold :
                false_negative_TDN += 1

    print(f'Number of positive : {positive} Number of negative : {negative}')
    print(f'For RESNET : False positive : {false_positive_classic} False negative : {false_negative_classic}')
    print(f'For CAM++ : False positive : {false_positive_CAM} False negative : {false_negative_CAM}')
    print(f'For TDN : False positive : {false_positive_TDN} False negative : {false_negative_TDN}')
    print(f'For RESNET : FPR : {100*(false_negative_classic/negative)}  : {100*(false_positive_classic/positive)}')
    print(f'For RESNET : EER Accuracy : {(100*(false_negative_classic/negative)+100*(false_positive_classic/positive))/2}')
    print(f'For CAM++ : EER Accuracy : {100*((false_negative_CAM/negative)+(false_positive_CAM/positive))/2}')   
    print(f'For TDN : EER Accuracy : {100*((false_negative_TDN/negative)+(false_positive_TDN/positive))/2}')

    false_positive_classic = 0
    false_positive_CAM = 0
    false_negative_classic = 0
    false_negative_CAM = 0
    false_positive_TDN = 0
    false_negative_TDN = 0
    positive = 0
    negative = 0

********************** CHOSEN TRESHOLD :  0.1  **********************
Number of positive : 2100 Number of negative : 18000
For RESNET : False positive : 228 False negative : 9530
For CAM++ : False positive : 85 False negative : 7994
For TDN : False positive : 228 False negative : 6015
For RESNET : FPR : 52.94444444444445  : 10.857142857142858
For RESNET : EER Accuracy : 31.900793650793652
For CAM++ : EER Accuracy : 24.22936507936508
For TDN : EER Accuracy : 22.136904761904763
********************** CHOSEN TRESHOLD :  0.15  **********************
Number of positive : 2100 Number of negative : 18000
For RESNET : False positive : 379 False negative : 6127
For CAM++ : False positive : 183 False negative : 4751
For TDN : False positive : 381 False negative : 2882
For RESNET : FPR : 34.03888888888889  : 18.047619047619047
For RESNET : EER Accuracy : 26.04325396825397
For CAM++ : EER Accuracy : 17.554365079365077
For TDN : EER Accuracy : 17.07698412698413
********************** CHOSEN TRESHOL

In [59]:
test = []

path1 = '23456789876543'
path2 = '12345678987654'
path3 = '12345678937654'



In [61]:
test.append({path1, path2})

In [64]:
if {path2, path1} in test :
    print('yes')

yes
